# run-zurich-validation.sh

Inside the `run-zurich-validation.sh` script, we have a few different test scenarios that we want to run:`validate-data`, `validate-machine-status`, and `validate-total-counter`. In this notebook we will be able to run through each of these tests and see the individualized results. 

For the purposes of this notebook, we will skip this cell and run the tests individually.

In [1]:
# to run full integration tests
./run-zurich-validation.sh

 => Running input watcher on zurich/input 
Setting up watches.
Watches established.


 => Running output watcher on metrics/aio/total-count 
 => Running output watcher on zurich/input/valid 
 => Running output watcher on metrics/aio/machine-status 
 => Running output watcher on zurich/input/invalid 
 => Running output watcher on zurich/debug 
 => Running output watcher on zurich/input/valid 
 => Running output watcher on metrics/aio/machine-status 
 => Running output watcher on zurich/input/invalid 
 => Running output watcher on zurich/debug 
Processing ./assets/zurich-validation/data-validation/invalid/invalid-empty-payload.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-messy-payload-01.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-messy-payload-02.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-missing-messageType.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-type-messageType.json
Processing ./assets/zurich-validation/data-validation/valid/valid-01.json
Pro

## Running the Tests

This first section sets up the test environment and creates the watchers needed to run the tests. 

Once this is run you should see a new directory called `.workspace` which contains the `logs`, `metrics`, and `zurich` directories with the various outputs in json files. The `logs` directory contains the logs for the tests, the `metrics` directory contains the machine status and total counter topic outputs, and the `zurich` directory contains the input and processed input json files for the tests.

In [4]:
#!/bin/bash

. ../utils/set-colors.sh
. ../utils/create-workspace.sh
. ./data-feeder/run-input-watchers.sh
. ./data-feeder/run-output-watchers.sh
. ./validations/validate-data.sh
. ./validations/validate-machine-status.sh
. ./validations/validate-total-counter.sh

declare -A topics_and_logs
topics_and_logs=(
  ["zurich/input/valid"]="zurich-valid.log"
  ["zurich/input/invalid"]="zurich-invalid.log"
  ["zurich/debug"]="zurich-debug.log"
  ["metrics/aio/machine-status"]="zurich-aio-machine-status.log"
  ["metrics/aio/total-count"]="zurich-aio-total-count.log"
)

WORKSPACE=".workspace"
INPUT_TOPIC="zurich/input"
AIO_METRICS_TOPIC="metrics/aio"

create_workspace $WORKSPACE
run_input_watcher $INPUT_TOPIC
run_output_watchers

[1]   Terminated              ./data-feeder/run-input.sh "$1" > $WORKSPACE/logs/zurich-input.log
[2]   Terminated              ./data-feeder/run-output.sh "$topic" > "$WORKSPACE/logs/${topics_and_logs[$topic]}"
[3]   Terminated              ./data-feeder/run-output.sh "$topic" > "$WORKSPACE/logs/${topics_and_logs[$topic]}"
[4]   Terminated              ./data-feeder/run-output.sh "$topic" > "$WORKSPACE/logs/${topics_and_logs[$topic]}"
[5]-  Terminated              ./data-feeder/run-output.sh "$topic" > "$WORKSPACE/logs/${topics_and_logs[$topic]}"
[6]+  Terminated              ./data-feeder/run-output.sh "$topic" > "$WORKSPACE/logs/${topics_and_logs[$topic]}"


 => Running input watcher on zurich/input 
[1] 150810
Setting up watches.
Watches established.
[1] 150810
Setting up watches.
Watches established.
 => Running output watcher on metrics/aio/total-count 
[2] 150846
 => Running output watcher on zurich/input/valid 
[3] 150847
 => Running output watcher on metrics/aio/machine-status 
[4] 150848
 => Running output watcher on zurich/input/invalid 
[5] 150849
 => Running output watcher on zurich/debug 
[6] 150850


Next we will run the `validate-data` test. This will iterate over the input files in the `/assets/zurich-validation/data-validation` directory and validate the data. We have 2 types of inputs: `valid-*.json` and `invalid-*.json`. The `valid-*.json` files are correct input payloads and should pass the validation. The `invalid-*.json` files should fail the validation based on various issues with the payload.

In [5]:
# test for valid input data
test_for_valid_data

Processing ./assets/zurich-validation/data-validation/invalid/invalid-empty-payload.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-messy-payload-01.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-messy-payload-02.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-messy-payload-01.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-messy-payload-02.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-missing-messageType.json
Processing ./assets/zurich-validation/data-validation/invalid/invalid-type-messageType.json
Processing ./assets/zurich-validation/data-validation/valid/valid-01.json
Processing ./assets/zurich-validation/data-validation/valid/valid-02.json
Processing ./assets/zurich-validation/data-validation/valid/valid-03.json
 => [EXPECTED] Valid: 3, Invalid: 5 
 => Waiting for processing [8 seconds] ... 

 => [TEST RESULTS] 

 INFO: 3 messages found 
 INFO:

The next test is the `total-counter` test. This test will iterate over the input files in the `/assets/zurich-validation/total-counter` directory and validate the data. It will pass in various payloads to the total-counter pipeline and check the output to make sure it is correct.

In [6]:
# test for total counter scenarios
# Scenario 1: total count = 1
# Scenario 2: total count 150
# Scenario 3: total count = 250 with multiple shifts

test_for_total_counter "$AIO_METRICS_TOPIC"

Processing ./assets/zurich-validation/start-shift-total-count/start-shift-good-counter-machine2.json
Processing ./assets/zurich-validation/start-shift-total-count/start-shift-bad-counter-machine2.json
Processing ./assets/zurich-validation/total-counter/total-count-1/good-counter-1-machine2.json
 Current AIO count: 1123 (from: .workspace/metrics/aio/total-count) 
Processing ./assets/zurich-validation/start-shift-total-count/start-shift-good-counter-machine2.json
Processing ./assets/zurich-validation/start-shift-total-count/start-shift-bad-counter-machine2.json
Processing ./assets/zurich-validation/total-counter/total-count-150-machine2/0-bad-counter-60-machine2.json
Processing ./assets/zurich-validation/total-counter/total-count-150-machine2/1-bad-counter-100-machine1.json
Processing ./assets/zurich-validation/total-counter/total-count-150-machine2/2-good-counter-90-machine2.json
 INFO: [.workspace/metrics/aio/total-count] 20240307232405101.json contains expected value of 1273. 
Process

Finally, we will test through the various machine status scenarios. This test will iterate over the input files in the `/assets/zurich-validation/machine-status` directory and validate the data. It will pass in various payloads to the machine-status pipeline and check the output to make sure it is correct. 

In [7]:
# test for machine status scenarios
# Scenario 1: machine status = "FAULT"
# Scenario 2: machine status = "IDLE"
# Scenario 3: machine status = "MODE1"
# Scenario 4: machine status = "MODE2"
# Scenario 5: machine status = "UNDEFINED"

test_for_machine_status "$AIO_METRICS_TOPIC"

Processing ./assets/zurich-validation/machine-status/fault/fault-true.json
 INFO: [.workspace/metrics/aio/machine-status] 20240307233333203.json contains expected value of FAULT. 
Processing ./assets/zurich-validation/machine-status/idle/fault-false.json
Processing ./assets/zurich-validation/machine-status/idle/idle-true.json
 INFO: [.workspace/metrics/aio/machine-status] 20240307233335263.json contains expected value of IDLE. 
Processing ./assets/zurich-validation/machine-status/mode1/fault-false.json
Processing ./assets/zurich-validation/machine-status/mode1/idle-false.json
Processing ./assets/zurich-validation/machine-status/mode1/mode1-true.json
Processing ./assets/zurich-validation/machine-status/mode1/mode2-false.json
 INFO: [.workspace/metrics/aio/machine-status] 20240307233339322.json contains expected value of MODE1. 
Processing ./assets/zurich-validation/machine-status/mode2/fault-false.json
Processing ./assets/zurich-validation/machine-status/mode2/idle-false.json
Processing

In order to rerun tests, you will need to stop background processes created from previous tests.

In [8]:
./clean-integration-tests.sh

pkill: killing pid 38103 failed: Operation not permitted
pkill: killing pid 38105 failed: Operation not permitted
pkill: killing pid 38205 failed: Operation not permitted
pkill: killing pid 38208 failed: Operation not permitted
pkill: killing pid 40442 failed: Operation not permitted
pkill: killing pid 41809 failed: Operation not permitted
pkill: killing pid 42594 failed: Operation not permitted
pkill: killing pid 43638 failed: Operation not permitted


[1]   Terminated              ./data-feeder/run-input.sh "$1" > $WORKSPACE/logs/zurich-input.log
[2]   Terminated              ./data-feeder/run-output.sh "$topic" > "$WORKSPACE/logs/${topics_and_logs[$topic]}"
[3]   Terminated              ./data-feeder/run-output.sh "$topic" > "$WORKSPACE/logs/${topics_and_logs[$topic]}"
[4]   Terminated              ./data-feeder/run-output.sh "$topic" > "$WORKSPACE/logs/${topics_and_logs[$topic]}"
[5]-  Terminated              ./data-feeder/run-output.sh "$topic" > "$WORKSPACE/logs/${topics_and_logs[$topic]}"
[6]+  Terminated              ./data-feeder/run-output.sh "$topic" > "$WORKSPACE/logs/${topics_and_logs[$topic]}"


: 1